In [1]:
!pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [2]:
import json

In [8]:
with open('annotations .json','r') as f:
  data=json.load(f)

In [9]:
len(data['annotations'])

197

In [10]:
data['annotations'] = [item for item in data['annotations'] if item is not None]

In [ ]:
train_data=data['annotations'][:(len(data['annotations'])//10)*8]

In [ ]:
test_data=data['annotations'][(len(data['annotations'])//10)*8:]

In [ ]:
len(train_data)

104

In [ ]:
len(test_data)

27

In [ ]:
!pip install -U spacy -q

In [ ]:
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
nlp=spacy.blank("en")
db=DocBin()

In [ ]:
data['annotations']

[['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
  {'entities': [[13, 17, 'PERSON'], [71, 77, 'GPE']]}],
 ['Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .',
  {'entities': [[0, 11, 'ORG'], [63, 84, 'QUANTITY'], [123, 167, 'SECTOR']]}],
 ['The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .',
  {'entities': [[17, 29, 'SECTOR'],
    [46, 53, 'ORG'],
    [66, 71, 'QUANTITY'],
    [94, 102, 'GPE'],
    [207, 217, 'ORG']]}],
 ['With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore i

In [ ]:
for text,annotations in tqdm(train_data):
  doc=nlp.make_doc(text)
  ents=[]
  for start,end,label in annotations['entities']:
    span=doc.char_span(start,end,label,alignment_mode="contract")
    if span is not None:
      ents.append(span)
  doc.ents=ents
  db.add(doc)


100%|██████████| 104/104 [00:00<00:00, 5766.59it/s]


In [ ]:
db.to_disk("./train_data.spacy")

In [ ]:
dbTest=DocBin()

In [ ]:
for text,annotations in tqdm(test_data):
  doc=nlp.make_doc(text)
  ents=[]
  for start,end,label in annotations['entities']:
    span=doc.char_span(start,end,label,alignment_mode="contract")
    if span is not None:
      ents.append(span)
  doc.ents=ents
  dbTest.add(doc)

dbTest.to_disk("./test_data.spacy")

100%|██████████| 27/27 [00:00<00:00, 4191.82it/s]


In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train_data.spacy --paths.dev ./train_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     56.14    0.28    0.80    0.17    0.00
  1     200         91.06   3389.93   85.87   85.82   85.92    0.86
  3     400         58.44   1235.89   95.36   94.71   96.02    0.95
  6     600         71.50    781.05   96.66   97.31   96.02    0.97
 10     800         54.10    545.23   98.74   98.23   99.25    0.99
 14    1000         77.87    473.31   97.74   98.36   97.11    0.98
 19    1200        127.47    546.96   98.25   97.45   99.08    0.98
 26    1400         97.58    489.19   99.11   98.30   99.94    0.99
 34    1600        100.33    546.

In [ ]:
trained_ner=spacy.load("model-best")

In [ ]:
text="In the first quarter of 2024, Infosys reported a net profit of EUR 1.2 billion, compared to EUR 950 million in the same period of 2023. The company’s revenue grew by 7.8% to EUR 4.3 billion, while operating margin improved from 19% to 21%. Analysts in Bengaluru noted that the growth was driven by strong demand for cloud services and AI products in the technology sector. Meanwhile, Tata Consultancy Services announced a dividend of EUR 0.25 per share, reflecting its continued financial stability and leadership in the IT services market."


In [ ]:
doc=trained_ner(text)

In [ ]:
spacy.displacy.render(doc,style="ent")

In [ ]:
text="""In the first quarter of 2024, Infosys, one of India’s largest IT services companies, reported a net profit of EUR 1.2 billion, compared to EUR 950 million in the same quarter of 2023. The company’s revenue grew by 7.8% to EUR 4.3 billion, while its operating margin improved from 19% to 21%. Analysts in Bengaluru noted that the growth was driven primarily by strong demand for cloud services, AI-driven products, and increased adoption of automation tools across the banking and retail sectors. CEO Salil Parekh highlighted that North America continued to be the strongest market, accounting for nearly 60% of the company’s total revenues.

Meanwhile, Tata Consultancy Services (TCS) announced its results for the fiscal year ending March 2024. The company reported revenue of EUR 27.5 billion, up 11% from EUR 24.7 billion a year earlier. Net profit rose 13% to EUR 7.2 billion, driven by contracts in the European financial sector and government digitization projects in the Middle East. The board also declared a final dividend of EUR 0.25 per share, payable in June 2024. According to CFO Samir Seksaria, TCS expects double-digit growth in fiscal 2025, fueled by demand in sectors such as manufacturing, energy, and telecommunications.

In the banking sector, ICICI Bank posted a net income of EUR 2.8 billion in the financial year 2023–2024, compared with EUR 2.1 billion in the previous year. Net interest margin improved from 4.1% to 4.3%, while total deposits grew 12% to EUR 150 billion. HDFC Bank, now merged with HDFC Ltd., reported even stronger results, with consolidated revenue of EUR 45.7 billion and net income of EUR 8.9 billion. The bank attributed this growth to higher retail lending, housing loan demand, and expansion in rural India. Analysts in Mumbai suggested that the merger created the largest private-sector lender in India, with significant potential to dominate the domestic market.

Globally, JP Morgan Chase in New York posted record quarterly earnings in Q1 2024, with net profit reaching USD 14.5 billion, up from USD 12.6 billion in Q1 2023. CEO Jamie Dimon noted that the bank benefited from strong investment banking fees, trading revenues, and higher interest income. In London, Barclays reported more modest results, with net income of GBP 3.2 billion, down from GBP 3.8 billion last year, citing weaker credit card demand and increased provisions for potential loan defaults. Meanwhile, Deutsche Bank in Frankfurt reported a 9% rise in revenue to EUR 8.7 billion, supported by asset management and corporate lending growth.

The energy sector also showed resilience. Reliance Industries, headquartered in Mumbai, reported consolidated revenue of INR 2.2 trillion for the quarter ending March 2024, with net profit of INR 180 billion, an increase of 14% year-on-year. Much of the growth came from the petrochemicals and refining business, although its telecom subsidiary Jio also contributed with subscriber growth to 480 million users. Saudi Aramco reported a net income of USD 121 billion for 2023, down from USD 161 billion in 2022, reflecting lower crude oil prices and OPEC+ production cuts. Still, the company declared a dividend of USD 31 billion, the largest quarterly payout in its history.

The technology sector globally is in transition. Apple announced iPhone 16 in September 2024 in Cupertino, California, with record pre-orders in the U.S., Europe, and India. Revenue from hardware products remained strong at USD 210 billion for the fiscal year, while services revenue including iCloud, Apple Music, and App Store fees reached USD 89 billion. Microsoft, headquartered in Redmond, Washington, reported revenue of USD 237 billion for fiscal 2024, with net profit of USD 85 billion. CEO Satya Nadella emphasized the role of AI tools such as Copilot in driving demand across enterprise customers. Google’s parent company Alphabet also posted strong results, with quarterly revenue of USD 74.6 billion and net profit of USD 19.3 billion in Q1 2024.

The pharmaceutical sector is also gaining attention. Pfizer reported revenue of USD 58 billion in 2023, down from USD 100 billion in 2022 due to declining COVID-19 vaccine sales. However, its oncology and rare disease portfolios grew by double digits. Novartis, based in Basel, Switzerland, recorded sales of USD 54 billion and net income of USD 12 billion, driven by new drug launches. In India, Dr. Reddy’s Laboratories posted revenue of INR 250 billion, supported by generic drug exports to the U.S. and Europe. Sun Pharma reported revenue of INR 430 billion, with net income of INR 75 billion, citing strong demand in the dermatology segment.

Manufacturing and automotive industries are also showing momentum. Tesla reported global deliveries of 1.9 million vehicles in 2023, with revenue of USD 97 billion and net profit of USD 15 billion. CEO Elon Musk announced that production of the Cybertruck would begin in Texas in mid-2024. Toyota, headquartered in Tokyo, Japan, reported revenue of JPY 37 trillion and net income of JPY 2.9 trillion for fiscal 2023, retaining its position as the world’s largest automaker by volume. Meanwhile, India’s Mahindra & Mahindra posted revenue of INR 720 billion, driven by SUV sales and its farm equipment division.

Commodity markets remained volatile. Gold prices averaged USD 1,950 per ounce in Q1 2024, up from USD 1,875 in Q4 2023, while crude oil traded between USD 70 and USD 85 per barrel. Copper demand remained strong, with prices averaging USD 8,600 per ton, driven by renewable energy and electric vehicle manufacturing. Agricultural commodities like wheat and corn also saw price spikes due to supply disruptions in Eastern Europe.

Global stock markets reflected these dynamics. On Wall Street, the S&P 500 gained 12% in the first half of 2024, while the Dow Jones Industrial Average rose 9%. In India, the benchmark Nifty 50 index on Dalal Street surged 18% year-to-date, supported by foreign institutional investor inflows of USD 9.3 billion. London’s FTSE 100 index rose 7%, while Tokyo’s Nikkei 225 gained 15%, reaching its highest level since 1989.

As the world moves into the second half of 2024, financial experts expect continued volatility. Rising interest rates in the U.S. and Europe, inflationary pressures in emerging markets, and geopolitical uncertainties in Eastern Europe and the Middle East could weigh on growth. However, structural drivers such as digital transformation, renewable energy adoption, and AI deployment across industries provide optimism for long-term stability. Investors remain focused on sectors like technology, healthcare, renewable energy, and infrastructure, expecting strong returns despite short-term fluctuations.
"""

In [ ]:
doc=trained_ner(text)

In [ ]:
spacy.displacy.render(doc,style="ent")

In [ ]:
!python -m spacy evaluate model-best ./test_data.spacy

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
NER P   100.00
NER R   99.42 
NER F   99.71 
SPEED   3574  


=============================== NER (per type) ===============================

                  P        R        F
ORG          100.00   100.00   100.00
FIN_METRIC   100.00   100.00   100.00
DATE         100.00   100.00   100.00
QUANTITY     100.00    97.96    98.97
GPE          100.00   100.00   100.00
PRODUCT      100.00   100.00   100.00
LOC          100.00   100.00   100.00
PERSON       100.00   100.00   100.00
SECTOR       100.00   100.00   100.00

